## ordinary differential equations

numerical integration
    
    gives numerical representation of a function in tabulated form in some dimension, integrates orbits
    (modeling stars and dark matter, calculating acceleration w a force equation)
    using high order approx of f(x) over interval [xi+1,xi] for integral F(a,b)
    
- trapezoid method (slide 6)
- simpson's rule (slide 7), treated as a parabola
- romberg integration (slide 8-11), until it reaches a tolerance, weighted sum times the width

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def func(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return a*x**2 + b*x + c

In [ ]:
def func_integral(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return (a*x**3)/3. + (b*x**2)/2. + c*x     #makes sure denominators are def as floats

trapezoid method

In [ ]:
#define the core, simplest part of the function that is testable (integral of trapezoid)

def trapezoid_core(f,x,h):
    return 0.5*h*(f(x+h) + f(x))     #f is function, x is lef side, h is right side (width)

In [ ]:
#define a wrapper (driving) function to perform trapezoid method

def trapezoid_method(f,a,b,N):
    #f = function to integrate
    #a = lower limit
    #b = upper limit
    #N = number of evaluations
    
    #define x values to perform trap method
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #define value of integral (F integer)
    Fint = 0.0
    
    #perform integral
    for i in range(0,len(x)-1,1):           
        Fint += trapezoid_core(f,x[i],h)    #prints (i,i+1,x[i],x[i]+h,x[-2],x[-1])
        
    return Fint

simpson's method (parabolic interpolation)

In [ ]:
def simpson_core(f,x,h):
    return h*( f(x) + 4*f(x+h) + f(x+2*h))/3.

In [ ]:
def simpsons_method(f,a,b,N):
    #if N is even, then the number of chunks is odd, we have to adjust the last segment
    
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    Fint = 0.0
    
    for i in range(0,len(x)-2,2):              #N-3 inclusive in steps of 2
        Fint += simpson_core(f,x[i],h)
        
    #apply simpson's rule over the last interval if N is even
    if((N%2)==0):
        Fint += simpson_core(f,x[-2],0.5*h)    #split the last interval in half

romberg integration

In [ ]:
def romberg_core(f,a,b,I,i):
    #I = array of integrations
    #i = current iteration
    
    #we need the difference btwn b-a
    h = b-a
    
    #increment btwn new func evals
    dh = h/2.**(i)
    
    #cofactor
    K = h/2.**(i+1)
    
    #and function evaluations
    M = 0.0
    for j in range(2**1):
        M += f(a + 0.5*dh + j*dh)  # func (far left + half current width + current width)
        
    #return answer
    return K*M

In [ ]:
#wrapper function

def romberg_integration(f,a,b,tol):
    
    #iteration variable
    i = 0
    
    #max number of iterations
    imax = 1000
    
    #error estimate, set to large value
    delta = 100.0*np.fabs(tol)         #iterate until tolerance, f absolute value
    
    #array of integral answers
    I = np.zeros(imax,dtype=float)     #array of imax, type float
    
    #get the zeroth romberh iteration
    I[0] = 0.5*(b-a)*(f(a) + f(b))
    
    #iterate by 1
    i += 1
    
    while(delta>tol):
        
        #while delta is larger than tolderance, we find this romberg iteration
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,I,i)
        
        #compute new fractional error estimate
        delta = np.fabs( (I[i]-I[i-1])/I[i] )
        
        print(i,I[i],I[i-1],delta)
        
        if(delta>tol):
            
            #iterate
            i += 1
            
            #if we reached max iterations 
            if (i>imax):
                print("Max iterations reached.")
                raise StopIteration('Stopping iterations after', i)
                
    return I[i]

In [ ]:
Answer = func_integral(1)-func_integral(0)
print(Answer)
print("trapezoid")
print(trapezoid_method(func,0,1,10))
print("simpson's method")
print(simpsons_method(func,0,1,10))
print("romberg")
tolerance = 1.0e-6
RI = romberg_integration(func,0,1,tolerance)
print(RI, (RI-Answer)/Answer, tolerance)